In [3]:
import librosa
import librosa.display
import os

import warnings
warnings.filterwarnings("ignore")

path_to_voice_records = "D:\\Utilisateurs\\ENAC\\projet_AIRBUS\\voice\\84\\121123\\"
path_to_airplane_records = "D:\\Utilisateurs\\ENAC\\projet_AIRBUS\\recordings\\"
path_to_created_samples = "D:\\Utilisateurs\\ENAC\\projet_AIRBUS\\created\\"

l_voice_records = os.listdir(path_to_voice_records)
l_airplane_records = os.listdir(path_to_airplane_records)
l_rec = l_voice_records + l_airplane_records

In [27]:
def mat_to_list(mat):
    out = []
    for x in mat:
        l_x = []
        for i in range(len(x)):
            l_x.append(x[i])
        out.append(l_x)
    return out

In [29]:
import random as rd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
n_train = 2

sp_voice = [] #spectres 
sp_plane = []
pca = PCA(0.99)

for i in range(n_train): 
    picked = rd.choice(l_voice_records) #pour l'instant que sur les voix
    signal,sample_rate = librosa.load(path_to_voice_records+picked)
    spectrum = librosa.stft(y=signal[:10000],center=False) #tronqué car les voix n'ont pas le meme temps d'enregistrement
    # print(len(spectrum),len(spectrum[0]))

    spectrum = spectrum.astype(float)
    pca.fit(spectrum)
    sp_voice.append(spectrum)

for i in range(n_train): 
    picked = rd.choice(l_airplane_records) 
    signal,sample_rate = librosa.load(path_to_airplane_records+picked)
    spectrum = librosa.stft(y=signal[:10000],center=False) #tronqué car les voix n'ont pas le meme temps d'enregistrement
    # print(len(spectrum),len(spectrum[0]))

    #On rescale la data
    spectrum = spectrum.astype(float)
    pca.fit(spectrum)
    sp_plane.append(spectrum)



In [34]:
len(pca.components_)

11

In [30]:
# len(sp_plane)

for i in range(n_train):
    sp_voice[i] = pca.transform(sp_voice[i])
    sp_plane[i] = pca.transform(sp_plane[i])

In [35]:
X_voice=[]
X_plane = []
for i in range(n_train):
    X_voice += mat_to_list(sp_voice[i])
    X_plane += mat_to_list(sp_plane[i])

In [32]:
len(X_voice),len(X_voice[0])

(2050, 11)

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

y_voice = [1 for elt in X_plane]
y_plane = [0 for elt in X_plane]

X = X_voice + X_plane
y = y_voice + y_plane


X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [37]:
from sklearn.metrics import accuracy_score

pred = rf.predict(X_test)
accuracy_score(pred,y_test)

0.8632668144863267